# Feature selection and feature engineering

This isn't based on a news article, exactly, it's from a paper. You can see the paper in `/data/`.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 200)
%matplotlib inline

# Reading in our data

## Every single person in a crash

We'll start by reading in the list of people who were involved in an accident.

**Call this dataframe `people`.**

In [2]:
people = pd.read_csv('data/combined-person-data.csv')

/Users/tbi/.pyenv/versions/3.6.5/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (5,15,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


How often did each severity of injury show up? (e.g. not injured, non-incapacitating injury, etc)

In [12]:
people.INJ_SEVER_CODE.value_counts()

1    716806
2     82642
3     76801
4     10353
5      1681
Name: INJ_SEVER_CODE, dtype: int64

We're only interested in fatalities, so let's create a new `had_fatality` column for when people received a fatal injury.

**Confirm there were 1681 people with fatal injuries.**

In [15]:
people['had_fatality'] = np.where(people.INJ_SEVER_CODE == 5, 1, 0)

In [16]:
people.had_fatality.sum()

1681

## Working on Features

### Starting our analysis

We're going to run a regression on the impact of being **male vs female on crash fatalities**. Prepare a dataframe called `train_df` with the appropriate information in it.

* **Tip:** What column(s) are your input, and what is your output? Aka independent and dependent variables
* **Tip:** You'll need to convert your input column into something numeric, I suggest using `.replace`
* **Tip:** We aren't interested in the "Unknown" sex - either filtering or `np.nan` + `.dropna()` might be useful ways to get rid of those columns

In [123]:
train_df = people[['had_fatality','SEX_CODE']].replace('F', 0).replace('M', 1).replace('U', np.nan).dropna(axis = 0)

Confirm that your `train_df` has two columns and 815,827 rows.

> **Tip:** If you have more rows, make sure you dropped all of the rows with Unknown sex.
>
> **Tip:** If you have more columns, make sure you only have your input and output columns.

In [124]:
train_df['SEX_CODE'] = train_df.SEX_CODE.astype(int)

In [125]:
train_df.shape

(815827, 2)

### Run your regression

See the effect of sex on whether the person's injuries are fatal or not. After we train the regression, we can use my ✨favorite technique✨ to display features and their odds ratios:
    
```python
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)
```

In [70]:
X = train_df.drop(columns="had_fatality")
y = train_df.had_fatality

clf = LogisticRegression(C=1e9, solver='lbfgs')
clf.fit(X,y)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
0,SEX_CODE,0.71481,2.0438


### Use words to interpret this result

In [ ]:
# if all other variables remain unchanged, being a man is 2 times more likely to die in car crushes than being a woman

## Adding more features

The actual crash data has more details - whether it was snowy/icy, whether it was a highway, etc. 

Read in `combined-crash-data.csv`, calling it **`crashes`**, and merge it with our people dataset. I'll save you a lookup: the `REPORT_NO` is what matches between the two.

In [95]:
crashes = pd.read_csv('data/combined-crash-data.csv')
crashes.sample(5)

,ACC_DATE,ACC_TIME,AGENCY_CODE,AREA_CODE,COLLISION_TYPE_CODE,COUNTY_NO,C_M_ZONE_FLAG,DISTANCE,DISTANCE_DIR_FLAG,FEET_MILES_FLAG,FIX_OBJ_CODE,HARM_EVENT_CODE1,HARM_EVENT_CODE2,JUNCTION_CODE,LANE_CODE,LATITUDE,LIGHT_CODE,LOC_CODE,LOGMILE_DIR_FLAG,LOG_MILE,LONGITUDE,MAINROAD_NAME,MUNI_CODE,RD_COND_CODE,RD_DIV_CODE,REFERENCE_NO,REFERENCE_ROAD_NAME,REFERENCE_SUFFIX,REFERENCE_TYPE_CODE,REPORT_NO,REPORT_TYPE,ROUTE_TYPE_CODE,RTE_NO,RTE_SUFFIX,SIGNAL_FLAG,SURF_COND_CODE,WEATHER_CODE
24433,2018-06-08 00:00:00,17:45:00,BACOPD,UNK,17,3.0,N,NaN,NaN,NaN,88.0,10.0,0.0,NaN,NaN,39.361353,1.00,NaN,NaN,NaN,-76.777900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AE5967003Q,Property Damage Crash,NaN,NaN,NaN,N,NaN,0.00
384677,2016-12-17 00:00:00,08:05:00,MSP,UNK,7,22.0,N,NaN,N,M,0.0,1.0,0.0,6.01,NaN,38.301777,6.02,NaN,N,0.55,-75.632866,SALISBURY BYPASS,0.0,1.0,4.0,13.0,S FRUITLAND BLVD,BU,US,MSP50530013,Injury Crash,US,13.0,NaN,N,4.0,9.04
12262,2018-05-19 00:00:00,02:36:00,HAGERSTOWN,UNK,88,21.0,N,200.0,N,F,0.0,2.0,2.0,0.00,1.0,39.650082,3.00,NaN,S,0.32,-77.720217,PROSPECT ST,73.0,1.0,2.0,1790.0,W NORTH AVE,NaN,MU,BX5253001K,Property Damage Crash,MU,2030.0,NaN,N,1.0,3.00
316607,03-01-2015,15:42:00,MSP,UNK,3,15.0,N,1.0,N,M,5.0,9.0,1.0,1.00,1.0,39.034161,1.00,NaN,N,4.14,-77.142765,EISENHOWER MEMORIAL HWY,NaN,1.0,4.0,927.0,MONTROSE RD,NaN,MD,MSP6271002X,NaN,IS,270.0,NaN,N,1.0,3.00
234599,2017-10-19 00:00:00,14:45:00,MSP,UNK,3,10.0,N,100.0,S,F,0.0,1.0,1.0,0.00,1.0,39.360897,1.00,NaN,N,7.87,-77.397681,EISENHOWER MEMORIAL HWY,0.0,1.0,4.0,0.0,CROSSOVER,NaN,UU,MSP6055005K,Injury Crash,IS,270.0,NaN,N,2.0,6.01


In [96]:
merged = people.merge(crashes, left_on = 'REPORT_NO', right_on = 'REPORT_NO')

### Examining more possible features

How often was it wet, dry, snowy, icy, etc? **What was the most common surfce condition?**

* **Tip:** We're interested in surface condition, _not_ road condition, _not_ weather condition

In [101]:
import re
names = '''
0	Not Applicable
1	Wet
2	Dry
3	Snow
4	Ice
5	Mud
6	Slush
7	Water 
8	Sand
9	Oil
88	Other
99	Unknown
'''

keys = [int(x) for x in re.findall('\d+', names)]
names = re.findall('([ A-Za-z]+)\n', names)
dict(zip(keys, names))

{0: 'Not Applicable',
 1: 'Wet',
 2: 'Dry',
 3: 'Snow',
 4: 'Ice',
 5: 'Mud',
 6: 'Slush',
 7: 'Water ',
 8: 'Sand',
 9: 'Oil',
 88: 'Other',
 99: 'Unknown'}

In [103]:
merged['SURF_COND_CODE'] = merged.SURF_COND_CODE.replace(dict(zip(keys, names)))

Do you feel that a **Dry** road condition should be the average of **Wet** and **Snow?**

In [ ]:
#Don't think so

The answer to that should be *no*, which means we can't use this data as numeric data. We want a different coefficient for each of these - I want to know the impact of dry, the impact of wet, the impact of snow, all separately.

Start by **replacing each code with a proper description.** I'll even include them here:

* `00` - Not Applicable
* `01`	- Wet
* `02`	- Dry
* `03`	- Snow
* `04`	- Ice
* `05`	- Mud, Dirt, Gravel
* `06`	- Slush
* `07`	- Water (standing/moving)
* `08`	- Sand
* `09`	- Oil
* `88`	- Other
* `99`	- Unknown

But watch out, pandas read the column in as numbers so they might have come through a little differently than their codes.

Confirm you have 147,803 wet, and a few codes you can't understand, like `6.03` and `7.01`.

In [105]:
merged.SURF_COND_CODE.value_counts()

Dry               619569
Wet               147803
Not Applicable     30921
Snow                9418
Ice                 9111
Unknown             3616
6.03                1697
Other               1064
Mud                  447
7.01                 258
9.88                 141
8.05                  36
Name: SURF_COND_CODE, dtype: int64

Replace the codes you don't understand with `Other`.

In [107]:
merged['SURF_COND_CODE'] = merged.SURF_COND_CODE.replace({6.03: 'Other', 7.01: 'Other', 9.88: 'Other', 8.05: 'Other'})


Confirm you have 3,196 'Other'.

In [110]:
(merged.SURF_COND_CODE == 'Other').sum()

3196

## One-hot encoding

We're going to use `pd.get_dummies` to build a variable you'll call `surf_dummies`. Each surface condition should be a `0` or `1` as to whether it was that condition (dry, icy, wet, etc).

Use a `prefix=` so we know they are **surface** conditions.

You'll want to drop the column you'll use as the reference category.

**Before we do this: which column works best as the reference?**

Now build your `surf_dummies` variable.

In [115]:
surf_dummies = pd.get_dummies(merged.SURF_COND_CODE, prefix = 'surf', drop_first = True)

In [116]:
surf_dummies.head()

,surf_Ice,surf_Mud,surf_Not Applicable,surf_Other,surf_Snow,surf_Unknown,surf_Wet
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1


Confirm your `surf_dummies` looks roughly like this:

|surface_Ice|Surce_Mud, Dirt, Gravel|surface_Not Applicable|...|surface_Wet|
|---|---|---|---|---|
|0|0|0|...|0|
|0|0|0|...|0|
|0|0|1|...|0|
|0|0|1|...|0|
|0|0|0|...|1|

## Another regression

Let's run another regression to see the impact of both **sex and surface condition** on fatalities.

### Build your `train_df`

To build your `train_df`, I recommend doing it either of these two ways. They both first select the important columns, then add in the one-hot encoded `surf_dummies` columns.

```python
train_df = pd.DataFrame({
    'col1': merged.col1,
    'col2': merged.col2,
    'col3': merged.col3,
})
train_df = train_df.join(surf_dummies)
train_df = train_df.dropna()
train_df.head()
```

or like this:

```python
train_df = train_df[['col1','col2','col3']].copy()
train_df = train_df.join(surf_dummies)
train_df = train_df.dropna()
train_df.head()
```

The second one is shorter, but the first one makes it easier to use comments to remove columns later.


In [159]:
train_df = merged[['SEX_CODE', 'had_fatality']].replace('F', 0).replace('M', 1).replace('U', np.nan).copy()

In [165]:
train_df = train_df.join(surf_dummies).dropna()

In [166]:
train_df.head()

,SEX_CODE,had_fatality,surf_Ice,surf_Mud,surf_Not Applicable,surf_Other,surf_Snow,surf_Unknown,surf_Wet
0,0.0,0,0,0,0,0,0,0,0
1,0.0,0,0,0,0,0,0,0,0
2,0.0,0,0,0,1,0,0,0,0
3,0.0,0,0,0,1,0,0,0,0
4,0.0,0,0,0,0,0,0,0,1


### Run your regression and check your odds ratios

Actually no, wait, first - what kind of surface do you think will have the **highest fatality rate?**

In [ ]:
# Snow

In [191]:
train_df.shape

(815843, 9)

In [193]:
train_df.head()

,SEX_CODE,had_fatality,surf_Ice,surf_Mud,surf_Not Applicable,surf_Other,surf_Snow,surf_Unknown,surf_Wet
0,0.0,0,0,0,0,0,0,0,0
1,0.0,0,0,0,0,0,0,0,0
2,0.0,0,0,0,1,0,0,0,0
3,0.0,0,0,0,1,0,0,0,0
4,0.0,0,0,0,0,0,0,0,1


Confirm your `train_df` has 815,843 rows and 9 columns.

* **Tip:** When you run your regression, if you get an error about not knowing what to do with `U`, it's because you didn't convert your sex to numbers (or if you did, you didn't do it in your original dataframe)

In [194]:
X = train_df.drop(columns = 'had_fatality')
y = train_df.had_fatality
clf = LogisticRegression(C=1e9, solver='lbfgs')
clf.fit(X, y)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [196]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
2,surf_Mud,1.118360,3.0598
0,SEX_CODE,0.718172,2.0507
4,surf_Other,0.596348,1.8155
6,surf_Unknown,0.340376,1.4055
7,surf_Wet,-0.098790,0.9059
1,surf_Ice,-0.558245,0.5722
5,surf_Snow,-0.653255,0.5203
3,surf_Not Applicable,-0.730477,0.4817


**Is this what you expected?** Why do you think this result might be the case?

In [ ]:
#Might be that wet, ice and snow could all be cause of muddy road

# More features: Vehicles

Maybe the car they're in is related to the car they were in. Luckily, we have this information - **read in `combined_vehicle_data` as `vehicles`.**

In [169]:
vehicles = pd.read_csv('data/combined-vehicle-data.csv')

## Weights of those cars

The car weights are stored in **another file** since the info had to come from an API. I looked up the VINs - vehicle identification numbers - in a government database to try to get data for each of them.

**Read them and build a new dataframe that is both the vehicle data along with their weights.** You can call it `vehicles` since you don't need the original weightless vehicle data any more.

In [170]:
weight = pd.read_csv('data/vins_and_weights.csv')

In [172]:
vehicles.head(2)

,AREA_DAMAGED_CODE1,AREA_DAMAGED_CODE2,AREA_DAMAGED_CODE3,AREA_DAMAGED_CODE_IMP1,AREA_DAMAGED_CODE_MAIN,BODY_TYPE_CODE,COMMERCIAL_FLAG,CONTI_DIRECTION_CODE,CV_BODY_TYPE_CODE,DAMAGE_CODE,DRIVERLESS_FLAG,FIRE_FLAG,GOING_DIRECTION_CODE,GVW_CODE,HARM_EVENT_CODE,HAZMAT_SPILL_FLAG,HIT_AND_RUN_FLAG,HZM_NUM,MOVEMENT_CODE,NUM_AXLES,PARKED_FLAG,REPORT_NO,SPEED_LIMIT,TOWED_AWAY_FLAG,TOWED_VEHICLE_CONFIG_CODE,VEHICLE_ID,VEH_MAKE,VEH_MODEL,VEH_YEAR,VIN_NO
0,8.0,9.0,10.0,10.0,10.0,23.08,N,E,NaN,5,N,N,E,NaN,9.0,NaN,N,NaN,1.0,NaN,N,ADJ487004H,30,Y,0,000238fd-44fa-4cd5-8eb7-41ab30500bec,CHEVY,TAHOE,2005.0,1GNEK13Q2J285593
1,12.0,NaN,NaN,12.0,12.0,2.00,N,E,NaN,4,N,N,E,NaN,1.0,NaN,N,NaN,1.0,NaN,N,MCP2487000M,40,Y,0,00038116-1bf9-48cc-b317-4f4375d14b60,INFI,4S,2003.0,JNKCV51E63M013580


In [173]:
weight.head(2)

,VIN,Make,Model,ModelYear,weight
0,2FMDA5143TBB45576,FORD,WINDSTAR,1996,3733.0
1,2G1WC5E37E1120089,CHEVROLET,IMPALA,2014,3618.0


In [175]:
vehicles = vehicles.merge(weight, left_on = 'VIN_NO', right_on = 'VIN')

Confirm that your combined `vehicles` dataset should have 534,436 rows and 35 columns. And yes, that's less than we were working with before - you haven't combined it with the people/crashes dataset yet.

In [176]:
vehicles.shape

(534436, 35)

### Filter your data

We only want vehicles that are "normal" - somewhere between 1500 and 6000 pounds. Filter your vehicles to only include those in that weight range.

In [180]:
vehicles = vehicles[(vehicles.weight >= 1500) & (vehicles.weight <= 6000)]

Confirm that you have 532,370 vehicles in the dataset.

In [181]:
vehicles.shape

(532370, 35)

## Add this vehicle information to your merged data

Now we'll have a dataframe that contains information on:

* The people themselves and their injuries
* The crash
* The vehicles

Every person came with a `VEHICLE_ID` column that is the vehicle they were in. You'll want to merge on that.

In [189]:
merged = merged.merge(vehicles, left_on = 'VEHICLE_ID', right_on = 'VEHICLE_ID')

Confirm you have 99 columns and 616,212 rows. **That is a lot of possible features!**

In [190]:
merged.shape

(616212, 99)

## Another regression, because we can't get enough

Build another `train_df` and run another regression about **how car weight impacts the chance of fatalities**. You'll want to confirm that your dataset has 616,212 and 2 columns.

In [292]:
train_df = merged[['had_fatality', 'weight']]

In [293]:
X = train_df.drop(columns = 'had_fatality')
y = merged.had_fatality
clf = LogisticRegression(C = 1e-2, solver = "lbfgs")

In [294]:
clf.fit(X, y)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [295]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)


,feature,coefficient (log odds ratio),odds ratio
0,weight,-0.000147,0.9999


**Can you translate that into plain English?** Remember weight is in **pounds**.

In [ ]:
# All else equal, every addtional one pound to the weight of car will be 1 times likely to cause fatality in car crush

# When the weight of car is increased by 1 pound, the odds of death in car crash increase by a factor or 1

I feel like pounds isn't the best measure for something like this. Remember how we had to adjust percentages with AP and life expecntancy, and then change around the way we said things? It sounded like this:

> Every 10% increase in unemployment translates to a year and a half loss of life expectancy

Instead of every single pound, maybe we could do every... some other number of pounds? One hundred? One thousand?

**Run another regression with weight in thousands of pounds.** Get another odds ratio. Give me another sentence English.

In [313]:
train_df['weight_in_thousand'] = train_df.copy().weight / 1000
X = train_df.drop(columns = ['had_fatality', 'weight'])
y = train_df.had_fatality

clf = LogisticRegression(C = 1e4, solver='lbfgs')
clf.fit(X, y)

/Users/tbi/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


LogisticRegression(C=10000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [314]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)


,feature,coefficient (log odds ratio),odds ratio
0,weight_in_thousand,-0.146881,0.8634


In [ ]:
# Every thousand pounds heavier your car is increase translates to a 15% decrease in fatalities

# Two-car accidents, struck and striker

Here's the thing, though: **it isn't just the weight of your car.** It's the weight of both cars! If I'm in a big car and I have a wreck with a smaller car, it's the smaller car that's in trouble.

To get that value, we need to do some **feature engineering**, some calculating of *new* variables from our *existing* variables.

We need to jump through some hoops to do that.

##  Two-car accidents

First we're going to count how many vehicles were in each accident. Since we're looking to compare the weight of two cars hitting each other, **we're only going to want crashes with only two cars.**

In [315]:
counted = vehicles.REPORT_NO.value_counts()
counted.head(10)

CB53480003      28
MDTA1229000H    26
CE4636002C      14
AS0286000B      14
MSP67460095     13
MSP6063006C     13
AE5607002W      12
MSP63070049     12
CC0237003W      12
DA3276000V      11
Name: REPORT_NO, dtype: int64

By using `.value_counts` I can see how many cars were in each crash, and now I'm going to filter to get a list of all of the ones with two vehicles.

In [316]:
two_car_report_nos = counted[counted == 2].index
two_car_report_nos

Index(['AC1288001Q', 'ADJ6960020', 'AE4890007B', 'CB5645004R', 'ADI5750002',
       'ADJ310000N', 'AE5948002Q', 'MCP1475001C', 'MSP6228000L', 'CTX308005S',
       ...
       'BK0261001H', 'MSP651500C5', 'MSP6485001M', 'ADJ632001J', 'MCP30290013',
       'MSP66710033', 'AE4721001C', 'AE5728002C', 'AJ0111000G', 'AE54750019'],
      dtype='object', length=137214)

And now we'll filter my vehicles so we only have those that were in two-vehicle crashes.

In [317]:
vehicles = vehicles[vehicles.REPORT_NO.isin(two_car_report_nos)]

### Struck and striker

To do the math correctly, we need both the risk of someone dying in the smaller car _and_ the risk of someone dying in the bigger car. To do this we need to separate our cars into two groups:

* The 'struck' vehicle: did the person die inside?
* The 'striker' vehicle: how much heavier was it than the struck car?

But we don't know which car was which, so we have to try out both versions - pretending car A was the striker, then pretending car B was the striker. It's hard to explain, but you can read `Pounds That Kill - The External Costs of Vehicle Weight.pdf` for more details on how it works.

In [325]:
cars_1 = vehicles.drop_duplicates(subset='REPORT_NO', keep='first')
cars_2 = vehicles.drop_duplicates(subset='REPORT_NO', keep='last')

In [326]:
cars_merged_1 = cars_1.merge(cars_2, on='REPORT_NO', suffixes=['_striker', '_struck'])
cars_merged_2 = cars_2.merge(cars_1, on='REPORT_NO', suffixes=['_striker', '_struck'])
vehicles_complete = pd.concat([cars_merged_1, cars_merged_2])
vehicles_complete.head()

,AREA_DAMAGED_CODE1_striker,AREA_DAMAGED_CODE2_striker,AREA_DAMAGED_CODE3_striker,AREA_DAMAGED_CODE_IMP1_striker,AREA_DAMAGED_CODE_MAIN_striker,BODY_TYPE_CODE_striker,COMMERCIAL_FLAG_striker,CONTI_DIRECTION_CODE_striker,CV_BODY_TYPE_CODE_striker,DAMAGE_CODE_striker,DRIVERLESS_FLAG_striker,FIRE_FLAG_striker,GOING_DIRECTION_CODE_striker,GVW_CODE_striker,HARM_EVENT_CODE_striker,HAZMAT_SPILL_FLAG_striker,HIT_AND_RUN_FLAG_striker,HZM_NUM_striker,MOVEMENT_CODE_striker,NUM_AXLES_striker,PARKED_FLAG_striker,REPORT_NO,SPEED_LIMIT_striker,TOWED_AWAY_FLAG_striker,TOWED_VEHICLE_CONFIG_CODE_striker,VEHICLE_ID_striker,VEH_MAKE_striker,VEH_MODEL_striker,VEH_YEAR_striker,VIN_NO_striker,VIN_striker,Make_striker,Model_striker,ModelYear_striker,weight_striker,AREA_DAMAGED_CODE1_struck,AREA_DAMAGED_CODE2_struck,AREA_DAMAGED_CODE3_struck,AREA_DAMAGED_CODE_IMP1_struck,AREA_DAMAGED_CODE_MAIN_struck,BODY_TYPE_CODE_struck,COMMERCIAL_FLAG_struck,CONTI_DIRECTION_CODE_struck,CV_BODY_TYPE_CODE_struck,DAMAGE_CODE_struck,DRIVERLESS_FLAG_struck,FIRE_FLAG_struck,GOING_DIRECTION_CODE_struck,GVW_CODE_struck,HARM_EVENT_CODE_struck,HAZMAT_SPILL_FLAG_struck,HIT_AND_RUN_FLAG_struck,HZM_NUM_struck,MOVEMENT_CODE_struck,NUM_AXLES_struck,PARKED_FLAG_struck,SPEED_LIMIT_struck,TOWED_AWAY_FLAG_struck,TOWED_VEHICLE_CONFIG_CODE_struck,VEHICLE_ID_struck,VEH_MAKE_struck,VEH_MODEL_struck,VEH_YEAR_struck,VIN_NO_struck,VIN_struck,Make_struck,Model_struck,ModelYear_struck,weight_struck
0,6.0,7.0,NaN,6.0,6.0,20.0,N,N,NaN,3,N,N,N,NaN,1.0,NaN,N,NaN,6.0,NaN,N,CB5190006B,55,N,0,0003b659-2785-4868-8877-0b786a284827,TOYT,TK,2011.0,5TFUY5F1XBX167340,5TFUY5F1XBX167340,TOYOTA,TUNDRA,2011,5480.0,3.0,5.0,6.0,6.0,6.0,23.08,N,N,NaN,5,N,N,N,NaN,1.0,NaN,N,NaN,6.0,NaN,N,55,Y,0,0e7da302-5f53-45c8-8b28-393cdc38171d,JEEP,CHEROKEE,2015.0,1C4RJFBGXFC751955,1C4RJFBGXFC751955,JEEP,GRAND CHEROKEE,2015,4545.0
1,8.0,NaN,NaN,8.0,8.0,2.0,N,W,NaN,2,N,NaN,W,NaN,1.0,NaN,N,NaN,8.0,NaN,N,ADJ4590035,5,N,0,00050484-d08f-4b6e-bc7e-9ec270e94660,HONDA,CIVIC,2015.0,2HGFG4A59FH702545,2HGFG4A59FH702545,HONDA,CIVIC,2015,2754.0,99.0,NaN,NaN,99.0,99.0,2.00,N,W,NaN,99,N,N,W,NaN,1.0,NaN,Y,NaN,1.0,NaN,N,5,N,0,3ebfe7fc-4832-414d-a5e8-391b65a2f280,HONDA,ACCORD,2013.0,1HGCT2B81DA005695,1HGCT2B81DA005695,HONDA,ACCORD,2013,3163.0
2,12.0,NaN,NaN,12.0,12.0,2.0,N,N,NaN,4,N,NaN,N,NaN,0.0,NaN,N,NaN,1.0,NaN,N,ADJ849000Z,10,N,0,00057af4-d848-4cee-b854-707f57581f4e,HONDA,ACCORD,2003.0,1HGCM66313A037175,1HGCM66313A037175,HONDA,ACCORD,2003,3023.0,12.0,NaN,NaN,12.0,12.0,2.00,N,NaN,NaN,3,Y,NaN,NaN,NaN,1.0,NaN,N,NaN,10.0,NaN,Y,0,N,0,4cf496cc-f537-447e-bb8e-3ef48c8c3ddf,HONDA,TK,2010.0,5J6RE3H73AL008454,5J6RE3H73AL008454,HONDA,CR-V,2010,3389.0
3,1.0,11.0,12.0,12.0,12.0,2.0,N,S,NaN,4,N,N,S,NaN,1.0,NaN,N,NaN,12.0,NaN,N,AE5207008Z,30,Y,0,00089d4a-7038-4693-9e02-b402676631af,FORD,4D,2016.0,1FADP3K28GL258987,1FADP3K28GL258987,FORD,FOCUS,2016,2932.0,11.0,12.0,NaN,12.0,12.0,23.08,N,N,NaN,4,N,N,N,NaN,1.0,NaN,N,NaN,1.0,NaN,N,30,Y,0,1e69b284-719f-429b-baae-6ae4976a546b,BUICK,TK,2002.0,3G5DB03E52S528316,3G5DB03E52S528316,BUICK,RENDEZVOUS,2002,4024.0
4,2.0,NaN,NaN,2.0,2.0,2.0,N,S,NaN,99,N,N,S,NaN,1.0,NaN,Y,NaN,1.0,NaN,N,ADJ859000Y,15,N,0,0010076b-0a45-45f3-8796-f387b39cd85d,HONDA,CIVIC,2009.0,2HGFA16689H357624,2HGFA16689H357624,HONDA,CIVIC,2009,2678.0,10.0,NaN,NaN,10.0,10.0,2.00,N,NaN,NaN,3,Y,N,NaN,NaN,1.0,NaN,N,NaN,10.0,NaN,Y,15,N,0,b14273cb-b3e5-42c7-af59-df069e8c2f0f,HONDA,CIVIC,2011.0,2HGFG1B87BH517469,2HGFG1B87BH517469,HONDA,CIVIC,2011,2703.0


## Put people in their cars

Which car was each person in? We'll assign that now.

In [327]:
merged = people.merge(vehicles_complete, left_on='VEHICLE_ID', right_on='VEHICLE_ID_struck')
merged.head(3)

,AIRBAG_DEPLOYED,ALCOHOL_TESTTYPE_CODE,ALCOHOL_TEST_CODE,BAC_CODE,CDL_FLAG,CLASS,CONDITION_CODE,DATE_OF_BIRTH,DRUG_TESTRESULT_CODE,DRUG_TEST_CODE,EJECT_CODE,EMS_UNIT_LABEL,EQUIP_PROB_CODE,FAULT_FLAG,INJ_SEVER_CODE,LICENSE_STATE_CODE,MOVEMENT_CODE,OCC_SEAT_POS_CODE,PED_LOCATION_CODE,PED_OBEY_CODE,PED_TYPE_CODE,PED_VISIBLE_CODE,PERSON_ID,PERSON_TYPE,REPORT_NO_x,SAF_EQUIP_CODE,SEX_CODE,VEHICLE_ID,had_fatality,AREA_DAMAGED_CODE1_striker,AREA_DAMAGED_CODE2_striker,AREA_DAMAGED_CODE3_striker,AREA_DAMAGED_CODE_IMP1_striker,AREA_DAMAGED_CODE_MAIN_striker,BODY_TYPE_CODE_striker,COMMERCIAL_FLAG_striker,CONTI_DIRECTION_CODE_striker,CV_BODY_TYPE_CODE_striker,DAMAGE_CODE_striker,DRIVERLESS_FLAG_striker,FIRE_FLAG_striker,GOING_DIRECTION_CODE_striker,GVW_CODE_striker,HARM_EVENT_CODE_striker,HAZMAT_SPILL_FLAG_striker,HIT_AND_RUN_FLAG_striker,HZM_NUM_striker,MOVEMENT_CODE_striker,NUM_AXLES_striker,PARKED_FLAG_striker,REPORT_NO_y,SPEED_LIMIT_striker,TOWED_AWAY_FLAG_striker,TOWED_VEHICLE_CONFIG_CODE_striker,VEHICLE_ID_striker,VEH_MAKE_striker,VEH_MODEL_striker,VEH_YEAR_striker,VIN_NO_striker,VIN_striker,Make_striker,Model_striker,ModelYear_striker,weight_striker,AREA_DAMAGED_CODE1_struck,AREA_DAMAGED_CODE2_struck,AREA_DAMAGED_CODE3_struck,AREA_DAMAGED_CODE_IMP1_struck,AREA_DAMAGED_CODE_MAIN_struck,BODY_TYPE_CODE_struck,COMMERCIAL_FLAG_struck,CONTI_DIRECTION_CODE_struck,CV_BODY_TYPE_CODE_struck,DAMAGE_CODE_struck,DRIVERLESS_FLAG_struck,FIRE_FLAG_struck,GOING_DIRECTION_CODE_struck,GVW_CODE_struck,HARM_EVENT_CODE_struck,HAZMAT_SPILL_FLAG_struck,HIT_AND_RUN_FLAG_struck,HZM_NUM_struck,MOVEMENT_CODE_struck,NUM_AXLES_struck,PARKED_FLAG_struck,SPEED_LIMIT_struck,TOWED_AWAY_FLAG_struck,TOWED_VEHICLE_CONFIG_CODE_struck,VEHICLE_ID_struck,VEH_MAKE_struck,VEH_MODEL_struck,VEH_YEAR_struck,VIN_NO_struck,VIN_struck,Make_struck,Model_struck,ModelYear_struck,weight_struck
0,1.0,NaN,0.0,NaN,N,NaN,0.0,1985-05-28 00:00:00,NaN,0.0,0.0,NaN,0.0,N,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,166296bd-ffd3-4c16-aa74-4f4bf4139d8d,D,AB4313000X,13.0,F,b463eb20-2f01-4200-9d6f-b18888ce2593,0,3.0,NaN,NaN,3.0,3.0,2.0,N,E,NaN,2,N,NaN,E,NaN,1.0,NaN,Y,NaN,2.0,NaN,N,AB4313000X,25,N,0,3c8629d0-d524-47c1-bfbc-b18e07f3087e,LEXU,TK,2004.0,2T2HA31U24C031048,2T2HA31U24C031048,LEXUS,RX,2004,3900.0,7.0,NaN,NaN,7.0,7.0,2.00,N,S,NaN,2,N,N,S,NaN,1.0,NaN,N,NaN,6.0,NaN,N,25,N,0,b463eb20-2f01-4200-9d6f-b18888ce2593,CHEVY,TK,2007.0,2CNDL13F576252855,2CNDL13F576252855,CHEVROLET,EQUINOX,2007,3776.0
1,1.0,NaN,0.0,NaN,N,C,0.0,1960-10-04 00:00:00,NaN,0.0,0.0,NaN,0.0,Y,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,f3b2743f-fbc3-4345-9419-56a0ca29102c,D,AB4313000X,13.0,F,3c8629d0-d524-47c1-bfbc-b18e07f3087e,0,7.0,NaN,NaN,7.0,7.0,2.0,N,S,NaN,2,N,N,S,NaN,1.0,NaN,N,NaN,6.0,NaN,N,AB4313000X,25,N,0,b463eb20-2f01-4200-9d6f-b18888ce2593,CHEVY,TK,2007.0,2CNDL13F576252855,2CNDL13F576252855,CHEVROLET,EQUINOX,2007,3776.0,3.0,NaN,NaN,3.0,3.0,2.00,N,E,NaN,2,N,NaN,E,NaN,1.0,NaN,Y,NaN,2.0,NaN,N,25,N,0,3c8629d0-d524-47c1-bfbc-b18e07f3087e,LEXU,TK,2004.0,2T2HA31U24C031048,2T2HA31U24C031048,LEXUS,RX,2004,3900.0
2,1.0,NaN,0.0,NaN,N,C,0.0,1990-11-05 00:00:00,NaN,0.0,1.0,NaN,1.0,N,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,21fff0f9-ed13-4b87-ac04-29d018aff9d4,D,AB5218001Y,13.0,F,4dea42c3-e02c-4c6b-8ea0-c2f8c17f147f,0,10.0,11.0,12.0,11.0,11.0,2.0,N,N,NaN,3,N,N,N,NaN,1.0,NaN,N,NaN,1.0,NaN,N,AB5218001Y,25,N,0,057a3e63-3009-4d07-b956-78f53a0ca992,LINC,TOWNCAR,2003.0,1LNHM82W53Y644084,1LNHM82W53Y644084,LINCOLN,TOWN CAR,2003,4000.0,9.0,10.0,11.0,11.0,11.0,23.08,N,S,NaN,3,N,N,W,NaN,1.0,NaN,N,NaN,12.0,NaN,N,25,N,0,4dea42c3-e02c-4c6b-8ea0-c2f8c17f147f,HYUN,SW,2010.0,5NMSG3AB9AH400386,5NMSG3AB9AH400386,HYUNDAI,SANTA FE,2010,3900.0


# Add the crash details

You did this already! I'm going to do it for you. We're merging on `REPORT_NO_x` because there are so many `REPORT_NO` columns duplicated across our files that pandas started giving them weird names.

In [328]:
merged = merged.merge(crashes, left_on='REPORT_NO_x', right_on='REPORT_NO')
merged.head(3)

,AIRBAG_DEPLOYED,ALCOHOL_TESTTYPE_CODE,ALCOHOL_TEST_CODE,BAC_CODE,CDL_FLAG,CLASS,CONDITION_CODE,DATE_OF_BIRTH,DRUG_TESTRESULT_CODE,DRUG_TEST_CODE,EJECT_CODE,EMS_UNIT_LABEL,EQUIP_PROB_CODE,FAULT_FLAG,INJ_SEVER_CODE,LICENSE_STATE_CODE,MOVEMENT_CODE,OCC_SEAT_POS_CODE,PED_LOCATION_CODE,PED_OBEY_CODE,PED_TYPE_CODE,PED_VISIBLE_CODE,PERSON_ID,PERSON_TYPE,REPORT_NO_x,SAF_EQUIP_CODE,SEX_CODE,VEHICLE_ID,had_fatality,AREA_DAMAGED_CODE1_striker,AREA_DAMAGED_CODE2_striker,AREA_DAMAGED_CODE3_striker,AREA_DAMAGED_CODE_IMP1_striker,AREA_DAMAGED_CODE_MAIN_striker,BODY_TYPE_CODE_striker,COMMERCIAL_FLAG_striker,CONTI_DIRECTION_CODE_striker,CV_BODY_TYPE_CODE_striker,DAMAGE_CODE_striker,DRIVERLESS_FLAG_striker,FIRE_FLAG_striker,GOING_DIRECTION_CODE_striker,GVW_CODE_striker,HARM_EVENT_CODE_striker,HAZMAT_SPILL_FLAG_striker,HIT_AND_RUN_FLAG_striker,HZM_NUM_striker,MOVEMENT_CODE_striker,NUM_AXLES_striker,PARKED_FLAG_striker,REPORT_NO_y,SPEED_LIMIT_striker,TOWED_AWAY_FLAG_striker,TOWED_VEHICLE_CONFIG_CODE_striker,VEHICLE_ID_striker,VEH_MAKE_striker,VEH_MODEL_striker,VEH_YEAR_striker,VIN_NO_striker,VIN_striker,Make_striker,Model_striker,ModelYear_striker,weight_striker,AREA_DAMAGED_CODE1_struck,AREA_DAMAGED_CODE2_struck,AREA_DAMAGED_CODE3_struck,AREA_DAMAGED_CODE_IMP1_struck,AREA_DAMAGED_CODE_MAIN_struck,BODY_TYPE_CODE_struck,COMMERCIAL_FLAG_struck,CONTI_DIRECTION_CODE_struck,CV_BODY_TYPE_CODE_struck,DAMAGE_CODE_struck,DRIVERLESS_FLAG_struck,FIRE_FLAG_struck,GOING_DIRECTION_CODE_struck,GVW_CODE_struck,HARM_EVENT_CODE_struck,HAZMAT_SPILL_FLAG_struck,HIT_AND_RUN_FLAG_struck,HZM_NUM_struck,MOVEMENT_CODE_struck,NUM_AXLES_struck,PARKED_FLAG_struck,SPEED_LIMIT_struck,TOWED_AWAY_FLAG_struck,TOWED_VEHICLE_CONFIG_CODE_struck,VEHICLE_ID_struck,VEH_MAKE_struck,VEH_MODEL_struck,VEH_YEAR_struck,VIN_NO_struck,VIN_struck,Make_struck,Model_struck,ModelYear_struck,weight_struck,ACC_DATE,ACC_TIME,AGENCY_CODE,AREA_CODE,COLLISION_TYPE_CODE,COUNTY_NO,C_M_ZONE_FLAG,DISTANCE,DISTANCE_DIR_FLAG,FEET_MILES_FLAG,FIX_OBJ_CODE,HARM_EVENT_CODE1,HARM_EVENT_CODE2,JUNCTION_CODE,LANE_CODE,LATITUDE,LIGHT_CODE,LOC_CODE,LOGMILE_DIR_FLAG,LOG_MILE,LONGITUDE,MAINROAD_NAME,MUNI_CODE,RD_COND_CODE,RD_DIV_CODE,REFERENCE_NO,REFERENCE_ROAD_NAME,REFERENCE_SUFFIX,REFERENCE_TYPE_CODE,REPORT_NO,REPORT_TYPE,ROUTE_TYPE_CODE,RTE_NO,RTE_SUFFIX,SIGNAL_FLAG,SURF_COND_CODE,WEATHER_CODE
0,1.0,NaN,0.0,NaN,N,NaN,0.0,1985-05-28 00:00:00,NaN,0.0,0.0,NaN,0.0,N,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,166296bd-ffd3-4c16-aa74-4f4bf4139d8d,D,AB4313000X,13.0,F,b463eb20-2f01-4200-9d6f-b18888ce2593,0,3.0,NaN,NaN,3.0,3.0,2.0,N,E,NaN,2,N,NaN,E,NaN,1.0,NaN,Y,NaN,2.0,NaN,N,AB4313000X,25,N,0,3c8629d0-d524-47c1-bfbc-b18e07f3087e,LEXU,TK,2004.0,2T2HA31U24C031048,2T2HA31U24C031048,LEXUS,RX,2004,3900.0,7.0,NaN,NaN,7.0,7.0,2.00,N,S,NaN,2,N,N,S,NaN,1.0,NaN,N,NaN,6.0,NaN,N,25,N,0,b463eb20-2f01-4200-9d6f-b18888ce2593,CHEVY,TK,2007.0,2CNDL13F576252855,2CNDL13F576252855,CHEVROLET,EQUINOX,2007,3776.0,2018-04-19 00:00:00,15:50:00,ANNAPOLIS,UNK,3,2.0,N,0.0,S,F,0.0,1.0,0.0,0.0,1.0,38.976882,1.0,NaN,N,0.01,-76.486787,COMPROMISE ST,3.0,0.0,1.0,960.0,DUKE OF GLOUCHESTER ST,NaN,MU,AB4313000X,Property Damage Crash,MU,740.0,NaN,N,0.0,7.01
1,1.0,NaN,0.0,NaN,N,C,0.0,1960-10-04 00:00:00,NaN,0.0,0.0,NaN,0.0,Y,1,MD,NaN,NaN,NaN,NaN,NaN,NaN,f3b2743f-fbc3-4345-9419-56a0ca29102c,D,AB4313000X,13.0,F,3c8629d0-d524-47c1-bfbc-b18e07f3087e,0,7.0,NaN,NaN,7.0,7.0,2.0,N,S,NaN,2,N,N,S,NaN,1.0,NaN,N,NaN,6.0,NaN,N,AB4313000X,25,N,0,b463eb20-2f01-4200-9d6f-b18888ce2593,CHEVY,TK,2007.0,2CNDL13F576252855,2CNDL13F576252855,CHEVROLET,EQUINOX,2007,3776.0,3.0,NaN,NaN,3.0,3.0,2.00,N,E,NaN,2,N,NaN,E,NaN,1.0,NaN,Y,NaN,2.0,NaN,N,25,N,0,3c8629d0-d524-47c1-bfbc-b18e07f3087e,LEXU,TK,2004.0,2T2HA31U24C031048,2T2HA31U24C031048,LEXUS,RX,2004,3900.0,2018-04-19 00:00:00,15:50:00,ANNAPOLIS,UNK,3,2.0,N,0.0,S,F,0.0,1.0,0.0,0.0,1.0,38.976882,1.0,NaN,N,0.01,-76.486787,COMPROMISE ST,3.0,0.0,1.0,960.0,DUKE OF GLOUCHESTER ST,NaN,MU,AB4313000X,Property Damage Crash,MU,740.0,NaN,N,0.0,7.01
2,1.0,NaN,0.0,

## Filter

We already filtered out vehicles by weight, so we don't have to do that again.

# Calculated features

I'm sure you forgot what all the features are, so we'll bring back whether there was a fatality or not

## Feature: Accident was fatal

In [329]:
merged['had_fatality'] = (merged.INJ_SEVER_CODE == 5).astype(int)
merged.had_fatality.value_counts()

0    334152
1       244
Name: had_fatality, dtype: int64

## Feature: Weight difference

**Remove everything missing weights for strikers or struck vehicles.** You might need to `merged.columns` to remind yourself what the column names are.

In [334]:
merged = merged.dropna(subset= ['weight_striker', 'weight_struck'])


Confirm your dataset has roughly 335,000 rows.

In [335]:
merged.shape

(334396, 135)

Create a new feature called `weight_diff` about how much heavier the striking car was compared to the struck car. **Make sure you've done the math correctly!**

In [339]:
merged['weight_diff'] = merged.weight_striker - merged.weight_struck

### Feature adjustment

Make all of your weight columns in **thousands of pounds** instead of just in pounds. It'll help you interpret your results much better.

In [343]:
merged[['weight_striker', 'weight_struck', 'weight_diff']] = merged[['weight_striker', 'weight_struck', 'weight_diff']] / 1000

In [345]:
merged[['weight_striker', 'weight_struck', 'weight_diff']].head()

,weight_striker,weight_struck,weight_diff
0,3.900,3.776,0.124
1,3.776,3.900,-0.124
2,4.000,3.900,0.100
3,3.900,4.000,-0.100
4,3.900,4.000,-0.100


# Another regression!!!

**What is the impact of weight difference on fatality rate?** Create your `train_df`, drop missing values, run your regression, analyze your odds ratios.

In [347]:
train_df = merged[['weight_diff', 'had_fatality']]

In [350]:
X = train_df.drop(columns = 'had_fatality')
y = train_df.had_fatality
clf = LogisticRegression(C = 1e4, solver = 'lbfgs')
clf.fit(X, y)

LogisticRegression(C=10000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [351]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
0,weight_diff,0.475533,1.6089


Please translate your odds ratio into plain English.

In [ ]:
# Every additional 1000 pounds in the weight differene between the striker and the struck car, the odds of death in car crash increases by a factor of 1.6
# All else equal, when there is a 1000 pounds increase in the weight differene between the striker and the struck car, it is 1.6 times more likely to have fatality in the car crush

## Adding in more features

How about speed limit? That's important, right? We can add the speed limit of the striking vehicle with `SPEED_LIMIT_striker`.

In [354]:
train_df = merged[['had_fatality', 'SPEED_LIMIT_striker']]

In [360]:
train_df['SPEED_LIMIT_striker'] = train_df.SPEED_LIMIT_striker / 5

/Users/tbi/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [361]:
X = train_df.drop(columns = 'had_fatality')
y = train_df.had_fatality

clf = LogisticRegression(C = 1e1, solver = 'lbfgs')
clf.fit(X, y)

LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [362]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
0,SPEED_LIMIT_striker,0.272272,1.3129


Can you translate the speed limit odds ratio into plain English?

In [ ]:
# Every addtional 5 mph increase in the striker's speed limit 

### Feature engineering: Speed limits

Honestly, that's a pretty bad way to go about things. What's more fun is if we **translate speed limits into bins.**

First, we'll use `pd.cut` to assign each speed limit a category.

In [364]:
speed_bins = [-np.inf, 10, 20, 30, 40, 50, np.inf]
merged['speed_bin'] = pd.cut(merged.SPEED_LIMIT_struck, bins=speed_bins)
merged[['SPEED_LIMIT_striker', 'speed_bin']].head(10)

,SPEED_LIMIT_striker,speed_bin
0,25,"(20.0, 30.0]"
1,25,"(20.0, 30.0]"
2,25,"(20.0, 30.0]"
3,25,"(20.0, 30.0]"
4,25,"(20.0, 30.0]"
5,5,"(-inf, 10.0]"
6,45,"(40.0, 50.0]"
7,45,"(40.0, 50.0]"
8,45,"(40.0, 50.0]"
9,45,"(40.0, 50.0]"


Then we'll one-hot encode around 20-30mph speed limits.

In [365]:
speed_dummies = pd.get_dummies(merged.speed_bin, 
                               prefix='speed').drop('speed_(20.0, 30.0]', axis=1)
speed_dummies.head()

,"speed_(-inf, 10.0]","speed_(10.0, 20.0]","speed_(30.0, 40.0]","speed_(40.0, 50.0]","speed_(50.0, inf]"
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


## Running a regression

I like this layout for creating `train_df`, it allows us to easily add dummies and do a little replacing/encoding when we're building binary features like for sex.

> If the below gives you an error, it's because `SEX_CODE` is already a number. In that case, just remove `.replace({'M': 1, 'F': 0, 'U': np.nan })`.

In [366]:
# Start with our normal features
train_df = pd.DataFrame({
    'weight_diff': merged.weight_diff,
#     'sex': merged.SEX_CODE,#.replace({'M': 1, 'F': 0, 'U': np.nan }),
    'had_fatality': merged.had_fatality,
})
# Add the one-hot encoded features
train_df = train_df.join(speed_dummies)
train_df = train_df.join(surf_dummies)
# Drop missing values
train_df = train_df.dropna()
train_df.head()

,weight_diff,had_fatality,"speed_(-inf, 10.0]","speed_(10.0, 20.0]","speed_(30.0, 40.0]","speed_(40.0, 50.0]","speed_(50.0, inf]",surf_Ice,surf_Mud,surf_Not Applicable,surf_Other,surf_Snow,surf_Unknown,surf_Wet
0,0.124,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-0.124,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.100,0,0,0,0,0,0,0,0,1,0,0,0,0
3,-0.100,0,0,0,0,0,0,0,0,1,0,0,0,0
4,-0.100,0,0,0,0,0,0,0,0,0,0,0,0,1


In [367]:
X = train_df.drop(columns = 'had_fatality')
y = train_df.had_fatality

In [370]:
clf = LogisticRegression(C = 1e9, solver = 'lbfgs')
clf.fit(X, y)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [371]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
4,"speed_(40.0, 50.0]",1.748303,5.7448
5,"speed_(50.0, inf]",1.443266,4.2345
3,"speed_(30.0, 40.0]",0.655175,1.9255
6,surf_Ice,0.637838,1.8924
0,weight_diff,0.466527,1.5944
9,surf_Other,0.367386,1.4440
8,surf_Not Applicable,0.236355,1.2666
12,surf_Wet,-0.131946,0.8764
10,surf_Snow,-0.175605,0.8389
1,"speed_(-inf, 10.0]",-1.068474,0.3435


Describe the impact of the different variables in simple language. What has the largest impact?

In [ ]:
# The speed limit ranging from 40 to 50 mph has the most impact on car crush fatality, meaning that with an increase of 20 in speed limit range, car crash is 5.7 times more likely to lead to death.

## Now you pick the features

Up above you have examples of:

* Creating features from numbers (speed limits)
* Creating features from 0/1 (sex)
* Creating features from binning numbers that are one-hot encoded (speed limit bins - `speed_bins`)
* Creating features from categories that are one-hot encoded (surface - `surf_dummies`

What else do you think matters? Try to plug in more features and see if you can get anything interesting.

> * **Hot tip:** The funniest/most interesting thing feature you can add is also the dumbest. Ask me about it in #algorithms if you end up getting down here.